In [1]:
import duckdb
import time
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

/home/newuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
con = duckdb.connect(database="tpch_sf1.db")

In [3]:
#2D
query_template = """select
	l_shipmode,
	sum(case
		when o_orderpriority = '1-URGENT'
			or o_orderpriority = '2-HIGH'
			then 1
		else 0
	end) as high_line_count,
	sum(case
		when o_orderpriority <> '1-URGENT'
			and o_orderpriority <> '2-HIGH'
			then 1
		else 0
	end) as low_line_count
from
	orders,
	lineitem
where
	o_orderkey = l_orderkey
	and l_shipmode in ('AIR', 'REG AIR')
	and l_commitdate < l_receiptdate
	and l_shipdate < l_commitdate
	and l_receiptdate >= '$receiptdate1'
	and l_receiptdate < '$receiptdate2'
group by
	l_shipmode
order by
	l_shipmode;"""


In [4]:
min_receiptdate = con.sql("SELECT MIN(l_receiptdate) FROM lineitem").fetchone()[0]
max_sreceiptdate = con.sql("SELECT MAX(l_receiptdate) FROM lineitem").fetchone()[0]
(min_receiptdate, max_sreceiptdate)

(datetime.date(1992, 1, 4), datetime.date(1998, 12, 31))

In [7]:
table = []

# Loop over the date range for `receiptdate`
for receiptdate1 in tqdm(list(pd.date_range(min_receiptdate, max_sreceiptdate, freq='30D'))):
    # Loop over the range for `receiptdate`
    for receiptdate2 in tqdm(list(pd.date_range(min_receiptdate, max_sreceiptdate, freq='30D')), leave=False):
            params = {'receiptdate1': receiptdate1, 'receiptdate2': receiptdate2}
            for key, val in params.items():
                query_template = query_template.replace(f"${key}", str(val))
            start = time.time()
            res = con.sql(query_template)
            elapsed = time.time() - start
            params['elapsed'] = elapsed
            table.append(params)

table = pd.DataFrame(table)
table

100%|██████████| 86/86 [00:01<00:00, 53.19it/s]


,receiptdate1,receiptdate2,elapsed
0,1992-01-04,1992-01-04,0.000448
1,1992-01-04,1992-02-03,0.000210
2,1992-01-04,1992-03-04,0.000170
3,1992-01-04,1992-04-03,0.000163
4,1992-01-04,1992-05-03,0.000241
...,...,...,...
7391,1998-12-28,1998-08-30,0.000167
7392,1998-12-28,1998-09-29,0.000175
7393,1998-12-28,1998-10-29,0.000178
7394,1998-12-28,1998-11-28,0.000159
